## Import packages

In [1]:
import pandas as pd
from CFModel import CFModel
from CFModel import DeepModel

Using TensorFlow backend.


## Define constants


In [2]:
RATINGS_CSV_FILE = './input/rating_result.csv'
USERS_CSV_FILE = './input/user_result.csv'
RECIPE_CSV_FILE = './input/recipe_result.csv'
MODEL_WEIGHTS_FILE = 'recipe_weights.h5'
K_FACTORS = 120
TEST_USER = 1000

## Load Recipe data

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'recipeid', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max() + 1
max_recipeid = ratings['recipeid'].drop_duplicates().max() + 1
print(len(ratings), 'ratings loaded.')

1048575 ratings loaded.


In [4]:
users = pd.read_csv(USERS_CSV_FILE, sep=',', encoding='utf-8', usecols=['userid', 'nickname'])
print(len(users), 'descriptions of', max_userid, 'users loaded.')

75061 descriptions of 12936 users loaded.


In [6]:
recipes = pd.read_csv(RECIPE_CSV_FILE, sep=',', encoding='utf-8', usecols=['recipeid', 'title', 'categories'])
print(len(recipes), 'descriptions of', max_recipeid, 'recipes loaded.')

5101 descriptions of 5023 recipes loaded.


## Make recommendations for a given user

In [7]:
trained_model = DeepModel(max_userid, max_recipeid, K_FACTORS)

In [8]:
trained_model.load_weights(MODEL_WEIGHTS_FILE)

In [9]:
users[users['userid'] == TEST_USER]

,userid,nickname
1000,1000,나리


In [10]:
def predict_rating(userid, recipeid):
    return trained_model.rate(userid, recipeid)

In [11]:
user_ratings = ratings[ratings['userid'] == TEST_USER][['userid', 'recipeid', 'rating']]

In [12]:
user_ratings['prediction'] = user_ratings.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
user_ratings.sort_values(by='rating', 
                         ascending=False).merge(recipes, 
                                                on='recipeid', 
                                                how='inner',
                                                suffixes=['_u', '_m']
                                               ).head(10)

,userid,recipeid,rating,prediction,title,categories
0,1000,2949,5,4.999999,초딩입맛이라면 누구나 다 좋아하는 맛살계란김밥! 맛살계란김밥,"['김밥/롤/주먹밥/초밥', '실생활 요리', '한식 요리', '달걀/유제품 요리'..."
1,1000,876,5,4.999998,집밥이 그리워 : 소고기 장조림과 깍두기를 넣은 장조림 버터밥 소고기장조림,"['쇠고기', '조림요리', '실생활 요리', '한식 요리', '움짤 레시피']"
2,1000,1104,5,4.999999,5분도 길다. 오늘은 1분 달달이! 설레오,"['여름', '아이스크림', '간식/야식', '퓨전요리', '5분 요리', '아이 ..."
3,1000,1080,5,4.999998,치즈덕후들을 일깨워 줄 오지치즈후라이,"['튀김요리', '요거트', '술안주', '양식 요리', '채소류 요리', '움짤 ..."
4,1000,1060,5,4.999999,화끈한 게 매력적이야 쏘핫쏘야 소시지야채볶음,"['햄/소시지', '볶음요리', '술안주', '퓨전요리', '움짤 레시피']"
5,1000,1034,5,5.000000,고소함과 짭짤함이 한입에 쏙~ 와인과 어울리는 참치카나페 참치카나페,"['수산가공품', '파티/명절요리', '양식 요리', '10분 요리']"
6,1000,1029,5,4.999997,"치즈성애자 심쿵하는, 치즈쭉쭉 고구마치즈피자 고구마피자","['피자', '간식/야식', '파티/명절요리', '퓨전요리', '달걀/유제품 요리'..."
7,1000,1010,5,4.999997,속재료 빵빵하게 채워 만든 치킨스낵랩 치킨스낵랩,"['닭고기', '햄버거/핫도그', '간식/야식', '멕시칸 요리', '간단 재료 요리']"
8,1000,991,5,4.999998,밀당과 사랑의 중심에서 와인을 외치다. '베이컨 꼬치' 베이컨 꼬치,"['햄/소시지', '구이요리', '술안주', '퓨전요리', '채소류 요리', '15..."
9,1000,984,5,4.999998,맛있는 찹스테이크 만들기 찹스테이크,"['쇠고기', '볶음요리', '술안주', '양식 요리']"


In [14]:
recommendations = ratings[ratings['recipeid'].isin(user_ratings['recipeid']) == False][['recipeid']].drop_duplicates()
recommendations['prediction'] = recommendations.apply(lambda x: predict_rating(TEST_USER, x['recipeid']), axis=1)
recommendations.sort_values(by='prediction',
                          ascending=False).merge(recipes,
                                                 on='recipeid',
                                                 how='inner',
                                                 suffixes=['_u', '_m']).head(10)

,recipeid,prediction,title,categories
0,801,5.000009,엄마가 만들어주는 스팸김밥&amp;참치롤 스팸 &amp; 참치 김밥,"['햄/소시지', '김밥/롤/주먹밥/초밥', '나들이 요리', '한식 요리', '1..."
1,1314,5.000008,달달함에 내가 솔로인 것도 잊는다! 국물떡볶이 떡볶이,"['볶음요리', '간식/야식', '한식 요리', '가공식품 요리', '자취 초보 요리']"
2,1620,5.000006,컵에 넣고 전자레인지에 땡! 치즈컵만두 치즈컵만두,"['만두요리', '간식/야식', '실생활 요리', '가공식품 요리', '5분 요리'..."
3,1078,5.000006,작다고 무시하지 마라! 작지만 강력한 레시피 땡초 김밥,"['김밥/롤/주먹밥/초밥', '실생활 요리', '한식 요리', '15분 요리']"
4,833,5.000006,"바쁜 아침, 그렇다고 거를 순 없지! 5분 밥버거 밥버거","['햄버거/핫도그', '나들이 요리', '퓨전요리', '가공식품 요리', '5분 요..."
5,1063,5.000005,머그컵으로 요리하기 베이컨에그,"['찜요리', '간식/야식', '양식 요리', '달걀/유제품 요리', '5분 요리'..."
6,1024,5.000005,사랑받는 남친이 되기 위한 스팸계란도시락 스팸계란도시락,"['햄/소시지', '김밥/롤/주먹밥/초밥', '나들이 요리', '한식 요리', '혼..."
7,747,5.000005,자꾸 손이 가는 식빵치즈스틱 식빵치즈스틱,"['구이요리', '튀김요리', '빵/케이크', '간식/야식', '술안주', '양식 ..."
8,1015,5.000004,국물이 없어도 맛있는 파송송 볶음라면! 볶음라면,"['한식면', '간식/야식', '한식 요리', '15분 요리', '볶음면요리']"
9,694,5.000004,나홀로 브런치 : 초간단 버섯 &amp; 베이컨 크림 리조또 버섯 &amp; 베이컨...,"['양식 요리', '달걀/유제품 요리', '자취 초보 요리', '혼밥 요리', '움..."
